### Next Word predictor!

Using LSTM, on a dataset of Sherlock Holmes' stories

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [3]:
import tensorflow as tf
from tensorflow import keras

2025-10-14 04:51:26.083248: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-14 04:51:26.593630: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-14 04:51:28.500522: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [6]:
text = open(r"/mnt/c/Rig/Pandora's BOX/DL/Datasets/1661-0.txt").read()
print(len(text))
print(text)

47369
﻿
Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle


Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release Date: November 29, 2002 [EBook #1661]
Last Updated: May 20, 2019

Language: English

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***



Produced by an anonymous Project Gutenberg volunteer and Jose Menendez



The Adventures of Sherlock Holmes



by Arthur Conan Doyle



Contents


   I.     A Scandal in Bohemia
   II.    The Red-Headed League
   III.   A Case of Identity
   IV.    The Boscombe Valley Mystery
   V.     The Five Orange Pips
   VI.    The Man with the Twisted Lip
   VII.   The Adventure of the Blue Carbuncle
   VIII.  The Adventure of the Speckled Band
   IX.    The Adventure of the Engineer’s Thumb



I. A SCANDAL IN BOHEMIA


I.

To Sherlock Holmes she is always _the_ woman. I have seldom heard him
mention her under any other name. In his eyes s

In [7]:
l = text.split('.')

In [5]:
print(l)

["\ufeff\nProject Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle\n\n\nTitle: The Adventures of Sherlock Holmes\n\nAuthor: Arthur Conan Doyle\n\nRelease Date: November 29, 2002 [EBook #1661]\nLast Updated: May 20, 2019\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\n*** START OF THIS PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***\n\n\n\nProduced by an anonymous Project Gutenberg volunteer and Jose Menendez\n\n\n\nThe Adventures of Sherlock Holmes\n\n\n\nby Arthur Conan Doyle\n\n\n\nContents\n\n\n   I", '     A Scandal in Bohemia\n   II', '    The Red-Headed League\n   III', '   A Case of Identity\n   IV', '    The Boscombe Valley Mystery\n   V', '     The Five Orange Pips\n   VI', '    The Man with the Twisted Lip\n   VII', '   The Adventure of the Blue Carbuncle\n   VIII', '  The Adventure of the Speckled Band\n   IX', '    The Adventure of the Engineer’s Thumb\n   X', '     The Adventure of the Noble Bachelor\n   XI', '    The Adventure of

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(l)

In [17]:
print(tokenizer.word_index)

{'the': 1, 'and': 2, 'of': 3, 'to': 4, 'a': 5, '”': 6, 'i': 7, 'in': 8, 'that': 9, 'it': 10, 'he': 11, 'was': 12, 'you': 13, 'his': 14, 'is': 15, 'have': 16, 'my': 17, 'as': 18, 'with': 19, 'which': 20, 'had': 21, 'at': 22, 'for': 23, 'not': 24, 'be': 25, 'me': 26, 'but': 27, 'from': 28, 'we': 29, 'upon': 30, 'holmes': 31, 'said': 32, 'there': 33, 'him': 34, 'this': 35, 'so': 36, 'on': 37, 'been': 38, 'by': 39, 'her': 40, 'all': 41, '“i': 42, 'one': 43, 'were': 44, 'are': 45, 'your': 46, 'very': 47, 'she': 48, 'would': 49, 'an': 50, 'out': 51, 'man': 52, 'then': 53, 'no': 54, 'what': 55, 'when': 56, 'up': 57, 'has': 58, 'into': 59, 'could': 60, 'some': 61, 'do': 62, 'mr': 63, 'will': 64, '’': 65, 'who': 66, 'little': 67, 'see': 68, 'down': 69, 'if': 70, 'now': 71, 'or': 72, 'our': 73, 'us': 74, 'am': 75, 'may': 76, 'know': 77, 'over': 78, 'must': 79, 'can': 80, 'they': 81, 'about': 82, 'more': 83, 'shall': 84, 'come': 85, 'than': 86, 'should': 87, 'did': 88, 'two': 89, 'other': 90, 'ro

In [9]:
print(tokenizer.word_counts['irene'])

11


In [10]:
inp_seq=[]
for sentence in l:
    tokenized_sentence = (tokenizer.texts_to_sequences([sentence])[0])

    for i in range(1,len(tokenized_sentence)):
        inp_seq.append(tokenized_sentence[:i+1])

print(inp_seq[1])
print(inp_seq[2])
print(inp_seq[3])

[810, 343, 811]
[810, 343, 811, 1]
[810, 343, 811, 1, 259]


In [11]:
maxlen = max(len(x) for x in inp_seq)
maxlen

80

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padding = pad_sequences(inp_seq, maxlen, padding='pre')

Since the last column has to be the target variable "Predicting the next word"

In [13]:
X = padding[:, :-1]
y = padding[:, -1]

In [14]:
print(X.shape)
print(y.shape)

(8401, 79)
(8401,)


In [15]:
input_dim = len(tokenizer.word_index) + 1

In [16]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes = input_dim)

In [17]:
y.shape

(8401, 2101)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [24]:
model = Sequential()

model.add(Embedding(input_dim=input_dim, output_dim=100, input_length=maxlen))
model.add(LSTM(64))       
model.add(Dense(input_dim, activation='softmax'))

In [25]:
model.build(input_shape=(None, 80))

In [26]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 80, 100)        │       210,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2101)           │       136,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 388,905 (1.48 MB)

 Trainable params: 388,905 (1.48 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
history = model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
callback = EarlyStopping(monitor = 'val_accuracy', patience=5)

In [28]:
model.fit(X,y,epochs = 50, callbacks=callback )

Epoch 1/50


2025-10-14 04:54:11.174963: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91200


263/263 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.0459 - loss: 6.5939
Epoch 2/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.0506 - loss: 6.1110
Epoch 3/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.0618 - loss: 5.9931
Epoch 4/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.0683 - loss: 5.8827
Epoch 5/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.0734 - loss: 5.7772
Epoch 6/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.0776 - loss: 5.6704
Epoch 7/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.0856 - loss: 5.5475
Epoch 8/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.0930 - loss: 5.4178
Epoch 9/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.1040 - loss: 5.2879
Epoch 10/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.1149 - loss: 5.1659
Epoch 11/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.1244 - loss: 5.0427
Epoch 12/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/ste

In [37]:
import time
text = "my deduction"

for i in range(20):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=maxlen, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))
  print(pos)  
  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
14
my deduction that
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
7
my deduction that i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
19
my deduction that i have
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
43
my deduction that i have been
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1019
my deduction that i have been caused
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
35
my deduction that i have been caused by
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
569
my deduction that i have been caused by someone
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
40
my deduction that i have been caused by someone who
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
63
my deduction that i have been caused by someone who has
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
564
my deduction that i have been caused by someone who has given
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
29
my deduction that i have been caused by someone who has given me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
44
my deduction that i have been caused by someone who ha